In [3]:
import pandas as pd
from gurobipy import *
xlsx = pd.ExcelFile('OR_final資料.xlsx')

# print(xlsx.sheet_names)

truck_sheet = pd.read_excel(xlsx, '消防車可服務火災種類')
# print(truck_sheet)

population_sheet = pd.read_excel(xlsx, '各區人口數')
# print(population_sheet)

fire_sheet = pd.read_excel(xlsx, '107-112火災各種類件數')
# print(fire_sheet)

distance_sheet = pd.read_excel(xlsx, '消防站和火災距離')
print(distance_sheet)


    距離(公里)          1          2          3          4          5          6  \
0        1   3.250497   4.855653   2.764086   2.110113   1.555134   1.870831   
1        2   2.737195   4.183701   2.020981   2.182538   1.434914   2.518396   
2        3   5.062221   5.921946   3.206512   4.699834   1.144735   4.235214   
3        4   4.714012   4.472885   2.379733   5.649345   3.087353   6.159977   
4        5   3.992621   5.589023   3.378330   2.647659   1.581857   1.760037   
5        6   6.310041   7.474935   4.811270   5.340970   2.368319   4.188397   
6        7   5.578143   6.974236   4.455138   4.336258   2.009837   3.048014   
7        8   5.703344   4.286747   4.129177   7.486801   5.962089   8.586342   
8        9   6.711303   6.026387   4.468018   7.820626   5.099194   8.292546   
9       10   7.082793   5.773006   5.271089   8.713685   6.709481   9.616411   
10      11   7.988331   6.873922   5.960409   9.409400   6.976595  10.086248   
11      12   3.309768   4.201955   1.537

泡沫消防車->0
乾粉消防車->1
水箱消防車->2

In [4]:
truck_type = truck_sheet.values.tolist()
cost = []
truck_support = []
for row in truck_type:
    # print(row)
    truck_support.append([row[1],row[2],row[3]])
    cost.append(row[4])
print(truck_support)
print(cost)

truck_type_num = len(truck_support)
print(truck_type_num)

[[1, 1, 0], [0, 1, 1], [1, 0, 0]]
[900, 1200, 700]
3


松山區->1
信義區->2
大安區->3
中山區->4
中正區->5
大同區->6
萬華區->7
文山區->8
南港區->9
內湖區->10
士林區->11
北投區->12


In [5]:
population_info = population_sheet.values.tolist()
population = []
for row in population_info:
    population.append(row[1])
print(population)

district_num = len(population)
print(district_num)

[193653, 206356, 291949, 216125, 150208, 119869, 172944, 260023, 113682, 275843, 266410, 242050]
12


普通火災->0	油類火災->1	電氣火災->2


In [6]:
fire_info = fire_sheet.values.tolist()
fire_type = []
for row in fire_info:
    fire_type.append([row[1], row[2], row[3]])
print(fire_type)

fire_type_num = len(fire_type[0])
print(fire_type_num)

[[176.0, 385.0, 247.0], [227.0, 414.0, 231.0], [237.0, 442.0, 366.0], [358.0, 472.0, 414.0], [156.0, 254.0, 269.0], [154.0, 196.0, 176.0], [281.0, 480.0, 307.0], [279.0, 520.0, 230.0], [119.0, 193.0, 120.0], [266.0, 475.0, 262.0], [369.0, 374.0, 344.0], [352.0, 337.0, 227.0]]
3


In [7]:
D = distance_sheet.drop(columns='距離(公里)').to_numpy()
print(D)

[[ 3.25049719  4.85565265  2.76408597  2.11011275  1.55513448  1.87083136
   2.56227238  8.08188524  8.3374198   6.87385576  5.17787983  9.64335158]
 [ 2.7371951   4.18370085  2.02098131  2.18253807  1.43491374  2.51839619
   2.90423573  7.42644047  7.75711858  6.56706403  5.5458206  10.18835217]
 [ 5.06222105  5.92194568  3.20651201  4.69983391  1.14473545  4.23521404
   1.5145936   6.66190851  9.70594523  9.04554415  7.78478359 11.97399639]
 [ 4.71401199  4.47288543  2.37973331  5.64934465  3.08735269  6.15997725
   4.41578527  3.86757185  8.09335823  8.55462284  9.15934374 13.88750291]
 [ 3.99262144  5.58902287  3.37833024  2.64765874  1.5818569   1.76003692
   1.99635111  8.4562581   9.08292292  7.5416593   5.34492465  9.53564312]
 [ 6.31004089  7.4749345   4.81126994  5.34097046  2.36831891  4.18839735
   0.71232448  8.28551175 11.2094973  10.12478034  7.8366047  11.43486287]
 [ 5.57814279  6.97423642  4.45513774  4.33625804  2.00983687  3.04801361
   0.61958596  8.68522446 10.609

In [8]:
import numpy as np
#distance_sheet = distance_sheet.drop(columns=['1/0'])
#covers = distance_sheet.values.tolist()
#covers = np.array(covers)
#print(covers)

station_num = len(D)
print(station_num)

46


In [29]:
ttl_P = 5000
P = []
for i in range(district_num):
    P.append([])
    for j in range(station_num):
        P[i].append(round(ttl_P * (1 - (D[j][i] / 20))))
print(np.array(P))

[[4187 4316 3734 3821 4002 3422 3605 3574 3322 3229 3003 4173 4471 4349
  3619 3407 4132 4489 4545 4308 4217 4513 4391 4426 4252 4722 4802 3420
  3480 3931 4244 4130 3958 3985 2726 2911 3130 3720 3775 3721 3689 2238
  2373 2073 2964 2010]
 [3786 3954 3520 3882 3603 3131 3256 3928 3493 3557 3282 3950 4539 4301
  3946 3811 4499 4991 4866 4687 3873 4110 3888 3998 3747 4744 4393 3655
  3540 4029 4238 3676 3537 3506 2358 2494 2624 3214 3293 3315 3189 1882
  1901 1568 2498 1535]
 [4309 4495 4198 4405 4155 3797 3886 3968 3883 3682 3510 4616 4766 4929
  3267 3144 3820 4312 4248 4434 3643 4528 4008 3884 3946 4330 4557 2989
  2949 3427 3708 4125 4065 3912 2195 2422 2883 3487 3366 3199 3359 1704
  1982 1841 2547 1634]
 [4472 4454 3825 3588 4338 3665 3916 3128 3045 2822 2648 4156 4050 4066
  3122 2876 3593 3922 3980 3788 4237 4583 4840 4509 4804 4158 4496 2988
  3178 3548 3874 4563 4350 4505 3001 3263 3685 4284 4228 3987 4213 2512
  2844 2627 3411 2492]
 [4611 4641 4714 4228 4605 4408 4498 3509 37

In [30]:
ttl_H = 50
H = []
for i in range(district_num):
    H.append([])
    for j in range(station_num):
        H[i].append(round(ttl_H * (1 - D[j][i] / 20)))
print(H)

[[42, 43, 37, 38, 40, 34, 36, 36, 33, 32, 30, 42, 45, 43, 36, 34, 41, 45, 45, 43, 42, 45, 44, 44, 43, 47, 48, 34, 35, 39, 42, 41, 40, 40, 27, 29, 31, 37, 38, 37, 37, 22, 24, 21, 30, 20], [38, 40, 35, 39, 36, 31, 33, 39, 35, 36, 33, 39, 45, 43, 39, 38, 45, 50, 49, 47, 39, 41, 39, 40, 37, 47, 44, 37, 35, 40, 42, 37, 35, 35, 24, 25, 26, 32, 33, 33, 32, 19, 19, 16, 25, 15], [43, 45, 42, 44, 42, 38, 39, 40, 39, 37, 35, 46, 48, 49, 33, 31, 38, 43, 42, 44, 36, 45, 40, 39, 39, 43, 46, 30, 29, 34, 37, 41, 41, 39, 22, 24, 29, 35, 34, 32, 34, 17, 20, 18, 25, 16], [45, 45, 38, 36, 43, 37, 39, 31, 30, 28, 26, 42, 41, 41, 31, 29, 36, 39, 40, 38, 42, 46, 48, 45, 48, 42, 45, 30, 32, 35, 39, 46, 43, 45, 30, 33, 37, 43, 42, 40, 42, 25, 28, 26, 34, 25], [46, 46, 47, 42, 46, 44, 45, 35, 37, 33, 33, 48, 41, 44, 27, 25, 32, 37, 37, 38, 34, 44, 39, 37, 40, 38, 42, 24, 25, 29, 32, 44, 45, 42, 21, 24, 32, 37, 34, 31, 35, 17, 22, 22, 27, 18], [45, 44, 39, 35, 46, 40, 42, 29, 29, 26, 25, 41, 37, 38, 26, 24, 31, 

Gurobi part:     i->district, j->station, k->fire truck type, r->fire type

In [31]:
eg1 = Model("eg1")

t = []
for j in range(station_num):
    t.append([])
    for k in range(truck_type_num):
        t[j].append(eg1.addVar(lb = 0, vtype = GRB.INTEGER, name = "t" + str(j+1) + str(k+1)))

eg1.update()

In [32]:
eg1.setObjective(quicksum(cost[k] * quicksum(t[j][k] for j in range(station_num)) for k in range(truck_type_num)), GRB.MINIMIZE)


for i in range(district_num):
    eg1.addConstr(quicksum(P[i][j] * quicksum(t[j][k] for k in range(truck_type_num)) for j in range(station_num)) >= population[i])

for i in range(district_num):
    for r in range(fire_type_num):
        eg1.addConstr(quicksum(H[i][j] * quicksum(truck_support[k][r] * t[j][k] for k in range(truck_type_num)) for j in range(station_num)) >= fire_type[i][r])


eg1.optimize()  

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 48 rows, 138 columns and 4416 nonzeros
Model fingerprint: 0x3b1f34c5
Variable types: 0 continuous, 138 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 5e+03]
  Objective range  [7e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+05]
Found heuristic solution: objective 950400.00000
Presolve removed 0 rows and 18 columns
Presolve time: 0.00s
Presolved: 48 rows, 120 columns, 3840 nonzeros
Found heuristic solution: objective 68900.000000
Variable types: 0 continuous, 120 integer (0 binary)

Root relaxation: objective 6.824683e+04, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf

In [33]:
if eg1.status == GRB.OPTIMAL:
    minCost = eg1.objVal
    print(minCost)
else:
    print("No optimal solution found. Model status:", eg1.status)

68900.0


In [34]:
# Prepare data for the DataFrame
data = []
total_trucks = 0
columns = ["Station"] + [f"Truck {k+1}" for k in range(truck_type_num)]

for j in range(station_num):
    row = [f"Station {j+1}"]
    for k in range(truck_type_num):
        truck_count = t[j][k].X
        row.append(truck_count)
        total_trucks += truck_count
    data.append(row)

# Create a DataFrame
df1 = pd.DataFrame(data, columns=columns)
print(df1)

print("Total Truck", total_trucks)

       Station  Truck 1  Truck 2  Truck 3
0    Station 1      0.0      0.0      0.0
1    Station 2      0.0      0.0      0.0
2    Station 3      0.0      0.0      0.0
3    Station 4      0.0      0.0      0.0
4    Station 5      0.0      0.0      0.0
5    Station 6      0.0      0.0      0.0
6    Station 7      0.0      0.0      0.0
7    Station 8      0.0      0.0      0.0
8    Station 9      0.0      0.0      0.0
9   Station 10      3.0      0.0      0.0
10  Station 11      0.0      0.0      0.0
11  Station 12      0.0      0.0      0.0
12  Station 13      0.0      0.0      0.0
13  Station 14      4.0      0.0     37.0
14  Station 15      0.0      0.0      0.0
15  Station 16      0.0      0.0      0.0
16  Station 17      0.0      0.0      0.0
17  Station 18      0.0      0.0      0.0
18  Station 19      0.0      0.0      0.0
19  Station 20      0.0      0.0      0.0
20  Station 21      0.0      0.0      0.0
21  Station 22      0.0      6.0      0.0
22  Station 23      0.0      1.0  

In [35]:
eg2 = Model("eg2")

t = []
for j in range(station_num):
    t.append([])
    for k in range(truck_type_num):
        t[j].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "t" + str(j+1) + str(k+1)))

w = []
for j in range(station_num):
    w.append(eg2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w" + str(j+1)))
    
e = eg2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "e")

eg2.update()

In [36]:
eg2.setObjective(e, GRB.MINIMIZE)

for j in range(station_num):
    eg2.addConstr(e >= quicksum(cost[k] * t[j][k] for k in range(truck_type_num)) - minCost / station_num)
    eg2.addConstr(w[j] == quicksum(cost[k] * t[j][k] for k in range(truck_type_num)) - minCost / station_num)

for j in range(station_num):
    eg2.addConstr(quicksum(t[j][k] for k in range(truck_type_num)) <= 5)
    
for i in range(district_num):
    eg2.addConstr(quicksum(P[i][j] * quicksum(t[j][k] for k in range(truck_type_num)) for j in range(station_num)) >= population[i])

for i in range(district_num):
    for r in range(fire_type_num):
        eg2.addConstr(quicksum(H[i][j] * quicksum(truck_support[k][r] * t[j][k] for k in range(truck_type_num)) for j in range(station_num)) >= fire_type[i][r])


eg2.optimize()  

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]


Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 186 rows, 185 columns and 4922 nonzeros
Model fingerprint: 0x0e994f99
Variable types: 47 continuous, 138 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 3e+05]
Found heuristic solution: objective 902.1739130
Presolve removed 0 rows and 46 columns
Presolve time: 0.00s
Presolved: 186 rows, 139 columns, 4876 nonzeros
Variable types: 0 continuous, 139 integer (0 binary)

Root relaxation: objective 1.370011e+02, 254 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  137.00108    0   87  902.17391  137.00108  84.8%     -    0s
H    0     0                     800.0000000  137.00108  82.9%     -    0s
H    0     0  

In [37]:
if eg2.status == GRB.OPTIMAL:
    max_gap = eg2.objVal
    print("Maximum gap:", max_gap)
    print("Budget per station:", minCost / station_num)
else:
    print("No optimal solution found. Model status:", eg2.status)

Maximum gap: 602.1739130434783
Budget per station: 1497.8260869565217


In [38]:
# Prepare data for the DataFrame
data = []
total_trucks = 0
columns = ["Station"] + [f"Truck {k+1}" for k in range(truck_type_num)] + ["Exceeding Expense"]

for j in range(station_num):
    row = [f"Station {j+1}"]
    for k in range(truck_type_num):
        truck_count = t[j][k].X
        row.append(truck_count)
        total_trucks += truck_count
    row.append(w[j].X)
    data.append(row)

# Create a DataFrame
df2 = pd.DataFrame(data, columns=columns)
print(df2)
# Write the DataFrame to an Excel file
excel_file = "firetruck_dispatcher_dis.xlsx"
df2.to_excel(excel_file, index=False)

print(f"Table saved to {excel_file}")
print("Maximum gap:", max_gap)
print("Budget per station:", minCost / station_num)
print("Total Truck", total_trucks)


       Station  Truck 1  Truck 2  Truck 3  Exceeding Expense
0    Station 1     -0.0     -0.0      3.0         602.173913
1    Station 2     -0.0     -0.0      3.0         602.173913
2    Station 3     -0.0      1.0      1.0         402.173913
3    Station 4     -0.0      1.0      1.0         402.173913
4    Station 5     -0.0     -0.0      3.0         602.173913
5    Station 6     -0.0     -0.0      3.0         602.173913
6    Station 7     -0.0     -0.0      3.0         602.173913
7    Station 8      1.0      1.0     -0.0         602.173913
8    Station 9     -0.0      1.0      1.0         402.173913
9   Station 10      1.0      1.0     -0.0         602.173913
10  Station 11      1.0      1.0     -0.0         602.173913
11  Station 12     -0.0     -0.0      3.0         602.173913
12  Station 13     -0.0     -0.0      3.0         602.173913
13  Station 14      1.0      1.0     -0.0         602.173913
14  Station 15      1.0      1.0     -0.0         602.173913
15  Station 16     -0.0 